In [13]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls

import pandas as pd 
import numpy as np

In [2]:
py.plotly.tools.set_credentials_file(username='savannahvi', api_key='tncaIB087gdSYQQgDQVZ')

## Prepare code

In [3]:
comic_data = pd.read_csv('comic_characters.csv')

In [4]:
comic_data['eyes'] = comic_data['eyes'].fillna('unknown')

In [5]:
comic_data['color'] = [1 if x == 'marvel' else 0 for x in comic_data['world']]

In [6]:
def encode_dict(col):
    '''
    creates a dictionary to encode the columns
    '''
    options = list(set(col))
    dict_opts = {}
    for i in range(len(options)):
        dict_opts[options[i]] = i
    return dict_opts


def encode(col, dic):
    return [x.replace(x,str(dic[x])) for x in col]

In [7]:
comic_data['align_id'] = encode(comic_data['align'], encode_dict(comic_data['align']))
comic_data['sex_id'] = encode(comic_data['sex'], encode_dict(comic_data['sex']))
comic_data['gsm_id'] = encode(comic_data['gsm'], encode_dict(comic_data['gsm']))
comic_data['alive_id'] = encode(comic_data['alive'], encode_dict(comic_data['alive']))

In [44]:
# adding randomness 
comic_data['align_id'] = [ float(x) + np.random.normal(0,.005) for x in comic_data['align_id']]
comic_data['sex_id'] = [ float(x) + np.random.normal(0,.005) for x in comic_data['sex_id']]
comic_data['gsm_id'] = [ float(x) + np.random.normal(0,.005) for x in comic_data['gsm_id']]
comic_data['alive_id'] = [ float(x) + np.random.normal(0,.005) for x in comic_data['alive_id']]

## Parallel Coordinate

In [45]:
data_comic = [
    go.Parcoords(
        line = dict(color = comic_data['color'],
                   colorscale = [[0,'#0000FF'],[1,'#FF0000']]),
        dimensions = list([
            dict(range = [1935, 2014],
                 constraintrange = [1935,2013],
                label = 'Year', values = comic_data['year']),
            dict(tickvals = [0,1,2,3],
                ticktext = ['Good Character', 'Neutral Character', 'Bad Character', 'Unknown'],
                label = 'Align', values = comic_data['align_id']),
            dict(tickvals = [0,1,2,3,4,5],
                 ticktext = ['Agender Chractres', 'Transgender Characters', 'Unknown', 'Male Character', 'Female Character', 'Genderfluid'],
                label = 'Sex', values = comic_data['sex_id']),
            dict(tickvals = [0,1,2,3,4,5,6],
                 ticktext = ['Transgender', 'Pansexual', 'Transvestite', 'Homosexual', 'Heterosexual', 'Bisexual', 'Genderfluid'],
                label = 'Gsm', values = comic_data['gsm_id']),
            dict(tickvals = [0, 1, 2],
                 ticktext = ['Living', 'Deceased', 'Unknown'],
                label = 'Alive', values = comic_data['alive_id']),
            dict(tickvals = [0, 1],
                 ticktext = ['DC', 'Marvel'],
                label = 'World', values = comic_data['color'])
        ])
    )
]



fig = go.Figure(data = data_comic)
py.iplot(fig, filename = 'parcoords-basic')

## Bar Charts for counts each year

In [9]:
comic_data['count'] = 1
df_align = comic_data[['align', 'year', 'count']].groupby(['align', 'year']).count().reset_index()
df_align.columns = ['align', 'year', 'align_count']

df_sex = comic_data[['sex', 'year', 'count']].groupby(['sex', 'year']).count().reset_index()
df_sex.columns = ['sex', 'year', 'sex_count']

df_gsm = comic_data[['gsm', 'year', 'count']].groupby(['gsm', 'year']).count().reset_index()
df_gsm.columns = ['gsm', 'year', 'gsm_count']

df_alive = comic_data[['alive', 'year', 'count']].groupby(['alive', 'year']).count().reset_index()
df_alive.columns = ['alive', 'year', 'alive_count']

In [10]:
# there are 4 needed colors for align
align_colors = ['#edf8e9', '#bae4b3', '#74c476', '#238b45']

# there are 6 needed colors for sex
sex_colors = ['#f2f0f7', '#dadaeb', '#bcbddc', '#9e9ac8', '#756bb1', '#54278f']

# there are 7 needed colors for gsm
gsm_colors = ['#fee5d9', '#fcbba1', '#fc9272', '#fb6a4a', '#ef3b2c', '#cb181d', '#99000d']

# there are 3 needed colors for alive
alive_colors = ['#deebf7', '#9ecae1', '#3182bd']

In [11]:
# creating list of options for each group 
alignment_list = list(set(df_align['align']))
sex_list = list(set(df_sex['sex'])) 
gsm_list = list(set(df_gsm['gsm'])) 
alive_list = list(set(df_alive['alive']))

In [12]:
fig = tls.make_subplots(rows=4, cols=1, shared_xaxes=True, subplot_titles=('Alignment', 'Sex',
                                                          'GSM', 'Alive'))
i = 0
for alignment in alignment_list:
    fig.append_trace({'x': df_align['year'], 'y': df_align[df_align['align'] == alignment]['align_count'], 'type': 'bar', 'name': 'align-{}'.format(alignment), 'marker': dict(color = align_colors[i])}, 1, 1)
    i += 1

j = 0
for sex in sex_list:
    fig.append_trace({'x': df_sex['year'], 'y': df_sex[df_sex['sex'] == sex]['sex_count'], 'type': 'bar', 'name': 'sex-{}'.format(sex), 'marker': dict(color = sex_colors[j])}, 2, 1)
    j = j + 1
      
k = 0
for gsm in gsm_list:
    fig.append_trace({'x': df_gsm['year'], 'y': df_gsm[df_gsm['gsm'] == gsm]['gsm_count'], 'type': 'bar', 'name': 'gsm-{}'.format(gsm), 'marker': dict(color = gsm_colors[k])}, 3, 1)
    k = k + 1

l = 0
for alive in alive_list:
    fig.append_trace({'x': df_alive['year'], 'y': df_alive[df_alive['alive'] == alive]['alive_count'], 'type': 'bar', 'name': 'alive-{}'.format(alive), 'marker': dict(color = alive_colors[l])}, 4, 1)
    l = l + 1
    
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



In [46]:
comic_data

,page_id,name,urlslug,id,align,eyes,hair,sex,gsm,alive,appearances,year,world,color,align_id,sex_id,gsm_id,alive_id,count
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,Heterosexual Characters,Living Characters,4043.0,1962.0,marvel,1,2.956945,1.054072,-0.088834,1.086564,1
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,Heterosexual Characters,Living Characters,3360.0,1941.0,marvel,1,2.950195,0.979439,-0.034410,1.123200,1
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,Heterosexual Characters,Living Characters,3061.0,1974.0,marvel,1,-0.001101,0.959099,-0.014785,0.995687,1
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,Heterosexual Characters,Living Characters,2961.0,1963.0,marvel,1,3.004469,0.959726,-0.033964,0.922364,1
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),Public Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Heterosexual Characters,Living Characters,2258.0,1950.0,marvel,1,3.033194,1.003368,-0.092407,0.897436,1
5,2458,Benjamin Grimm,\/Benjamin_Grimm_(Earth-616),Public Identity,Good Characters,Blue Eyes,Bald,Male Characters,Heterosexual Characters,Living Characters,2255.0,1961.0,marvel,1,2.991865,1.008030,0.007389,0.978230,1
6,2166,Reed Richards,\/Reed_Richards_(Earth-616),Public Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Heterosexual Characters,Living Characters,2072.0,1961.0,marvel,1,3.007782,0.979211,-0.104728,0.963278,1
7,1833,Hulk (Robert Bruce Banner),\/Hulk_(Robert_Bruce_Banner),Public Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,Heterosexual Characters,Living Characters,2017.0,1962.0,marvel,1,2.963262,0.993851,-0.075043,0.973944,1
8,29481,Scott Summers,\/Scott_Summers_(Earth-616),Public Identity,Neutral Characters,Brown Eyes,Brown Hair,Male Characters,Heterosexual Characters,Living Characters,1955.0,1963.0,marvel,1,0.112404,1.036955,-0.043344,1.002673,1
9,1837,Jonathan Storm,\/Jonathan_Storm_(Earth-616),Public Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,Heterosexual Characters,Living Characters,1934.0,1961.0,marvel,1,2.991302,0.949006,0.054735,1.042987,1
